# Run pipeline with bash 

You will need to install Salmon, fastQC and multiQC before running the commands below. Otherwise, you can create the conda environment, activate it and run the notebook then, as explained [here](create_conda_env.ipynb). 



### Set pipeline folder

In [ ]:
import os, shutil
pwd = os.getcwd()
os.getcwd()
pipeline_folder = "rnaseq-nf" 
work_dir = pwd + "/../" + pipeline_folder
os.chdir(work_dir)

### Create Salmon index

In [ ]:
!salmon index --threads 1 -t ./data/ggal/ggal_1_48850000_49020000.Ggal71.500bpflank.fa -i index

### Run Salmon and remove index

In [ ]:
!salmon quant --threads 1 --libType=U -i index -1 ./data/ggal/ggal_gut_1.fq -2 ./data/ggal/ggal_gut_2.fq  -o results_salmon
!rm -rf index

### Create fastQC results folder and run it

In [ ]:
if not os.path.exists("results_fastqc"):
    os.makedirs("results_fastqc")
    
!fastqc -o results_fastqc -f fastq -q ./data/ggal/ggal_gut_*.fq

### Create folders to move input and results for multiQC

In [ ]:
if not os.path.exists("multiqc_in"):
    os.makedirs("multiqc_in")

if not os.path.exists("results_multiqc"):
    os.makedirs("results_multiqc")

!cp results_fastqc/* multiqc_in
!cp -rf results_salmon/* multiqc_in
!cp multiqc/* multiqc_in
!echo "custom_logo: $PWD/multiqc_in/logo.png" >> multiqc_in/multiqc_config.yaml

pwd = os.getcwd()
work_dir = pwd + "/multiqc_in"
os.chdir(work_dir)

### Remove multiQC report if run before

In [ ]:
res_dir = pwd + "/results_multiqc"

try:
    shutil.rmtree(res_dir)
except OSError:
    pass

### Run multiQC

In [ ]:
!multiqc -v . --outdir ../results_multiqc/

### Remove input data for multiQC and calling firefox to display multiQC report 

In [ ]:
os.chdir(res_dir)
shutil.rmtree(work_dir)

### Uncomment to show the report on firefox, otherwise go to 
### /your_path/BovReg-Reproducibility/rnaseq-nf/results_multiqc and open the file multiqc_report.html
### with your favourite browser

# !firefox multiqc_report.html